# LangChain Implementation of Generative Agents: Interactive Simulacra of Human Behavior

In [ ]:
!pip install termcolor > /dev/null

In [1]:
import re
from datetime import datetime, timedelta
from typing import Any, Dict, List, Optional, Tuple
from termcolor import colored

from pydantic import BaseModel, Field

from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.schema import BaseLanguageModel, Document
from langchain.vectorstores import FAISS


## Key Components
- Observational loop
   - Need to connect to some world. Can simulate
- Memory - 2 types
   1. Observations - list/stack of observations
   2. Reflections - Syntheses over recent history
- Memory Retrieval
   - Weighted sum of recency and saliency
- Planning / Plan update

In [2]:
LLM = ChatOpenAI(max_tokens=1500) # Can be any LLM you want.

In [22]:
class GenerativeAgent(BaseModel):
    """A character with memory and innate characteristics."""
    
    name: str
    age: int
    traits: str
    """The traits of the character you wish not to change."""
    status: str
    """Current activities of the character."""
    llm: BaseLanguageModel
    memory_retriever: TimeWeightedVectorStoreRetriever
    """The retriever to fetch related memories."""
    verbose: bool = False
    
    reflection_threshold: Optional[float] = None
    """When the total 'importance' of memories exceeds the above threshold, stop to reflect."""

    summary: str = ""  #: :meta private:
    summary_refresh_seconds: int= 3600  #: :meta private:
    last_refreshed: datetime =Field(default_factory=datetime.now)  #: :meta private:
    daily_summaries: List[str] #: :meta private:
    memory_importance: float = 0.0 #: :meta private:
    
    class Config:
        """Configuration for this pydantic object."""

        arbitrary_types_allowed = True

    @staticmethod
    def _parse_list(text: str) -> List[str]:
        lines = re.split(r'\n', text.strip())
        return [re.sub(r'^\s*\d+\.\s*', '', line).strip() for line in lines]

    def _score_memory_importance(self, memory_content: str) -> float:
        """Score the absolute importance of the given memory."""
        prompt = PromptTemplate.from_template(
         "On the scale of 1 to 10, where 1 is purely mundane"
         +" (e.g., brushing teeth, making bed) and 10 is"
         + " extremely poignant (e.g., a break up, college"
         + " acceptance), rate the likely poignancy of the"
         + " following piece of memory. Respond with a single integer."
         + "\nMemory: {memory_content}"
         + "\nRating: "
        )
        chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        score = chain.run(memory_content=memory_content).strip()
        return float(score[0]) / 10

    def _compute_agent_summary(self):
        """"""
        prompt = PromptTemplate.from_template(
            "How would you summarize {name}'s core characteristics given the"
            +" following statements:\n"
            +"{related_memories}"
            + "Do not embellish."
            +"\n\nSummary: "
        )
        # The agent seeks to think about their core characteristics.
        relevant_memories = self.fetch_memories(f"{self.name}'s core characteristics")
        relevant_memories_str = "\n".join([f"{mem.page_content}" for mem in relevant_memories])
        chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        return chain.run(name=self.name, related_memories=relevant_memories_str).strip()
    
    def _get_topics_of_reflection(self, last_k: int = 100) -> Tuple[str, str, str]:
        """Return the 3 most salient high-level questions about recent observations."""
        prompt = PromptTemplate.from_template(
            "{observations}\n\n"
            + "Given only the information above, what are the 3 most salient"
            + " high-level questions we can answer about the subjects in the statements?"
            + " Provide each question on a new line.\n\n"
        )
        reflection_chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        observations = self.memory_retriever.memory_stream[-last_k:]
        observation_str = "\n".join([o.page_content for o in observations])
        result = reflection_chain.run(observations=observation_str)
        return self._parse_list(result)
    
    def _get_insights_on_topic(self, topic: str) -> List[str]:
        """Generate 'insights' on a topic of reflection, based on pertinent memories."""
        prompt = PromptTemplate.from_template(
            "Statements about {topic}\n"
            +"{related_statements}\n\n"
            + "What 5 high-level insights can you infer from the above statements?"
            + " (example format: insight (because of 1, 5, 3))"
        )
        related_memories = self.fetch_memories(topic)
        related_statements = "\n".join([f"{i+1}. {memory.page_content}" 
                                        for i, memory in 
                                        enumerate(related_memories)])
        reflection_chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        result = reflection_chain.run(topic=topic, related_statements=related_statements)
        # TODO: Parse the connections between memories and insights
        return self._parse_list(result)
    
    def add_memory(self, memory_content: str) -> List[str]:
        """Add an observation or memory to the agent's memory."""
        importance_score = self._score_memory_importance(memory_content)
        self.memory_importance += importance_score
        document = Document(page_content=memory_content, metadata={"importance": importance_score})
        result = self.memory_retriever.add_documents([document])
        if (self.reflection_threshold is not None 
            and self.memory_importance > self.reflection_threshold
            and self.status != "Reflecting"):
            old_status = self.status
            self.status = "Reflecting"
            self.pause_to_reflect()
            # Hack to clear the importance from reflection
            self.memory_importance = 0.0
            self.status = old_status
        return result
    
    def fetch_memories(self, observation: str) -> List[Document]:
        """Fetch related memories."""
        return self.memory_retriever.get_relevant_documents(observation)
    
        
    def get_summary(self, force_refresh: bool = False) -> str:
        """Return a descriptive summary of the agent."""
        current_time = datetime.now()
        since_refresh = (current_time - self.last_refreshed).seconds
        if not self.summary or since_refresh >= self.summary_refresh_seconds or force_refresh:
            self.summary = self._compute_agent_summary()
            self.last_refreshed = current_time
        return (
            f"Name: {self.name} (age: {self.age})"
            +f"\nInnate traits: {self.traits}"
            +f"\n{self.summary}"
        )
    
    def get_full_header(self, force_refresh: bool = False) -> str:
        """Return a full header of the agent's status, summary, and current time."""
        summary = self.get_summary(force_refresh=force_refresh)
        current_time_str =  datetime.now().strftime("%B %d, %Y, %I:%M %p")
        return f"{summary}\nIt is {current_time_str}.\n{self.name}'s status: {self.status}"

    def pause_to_reflect(self) -> List[str]:
        """Reflect on recent observations and generate 'insights'."""
        print(colored(f"Character {self.name} is reflecting", "blue"))
        new_insights = []
        topics = self._get_topics_of_reflection()
        for topic in topics:
            insights = self._get_insights_on_topic( topic)
            for insight in insights:
                self.add_memory(insight)
            new_insights.extend(insights)
        return new_insights
    
    def _get_entity_from_observation(self, observation: str) -> str:
        prompt = PromptTemplate.from_template(
            "What is the observed entity in the following observation? {observation}"
            +"\nEntity="
        )
        chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        return chain.run(observation=observation).strip()

    def _get_entity_action(self, observation: str, entity_name: str) -> str:
        prompt = PromptTemplate.from_template(
            "What is the {entity} doing in the following observation? {observation}"
            +"\nThe {entity} is"
        )
        chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        return chain.run(entity=entity_name, observation=observation).strip()
    
    def _format_memories_to_summarize(self, relevant_memories: List[Document]) -> str:
        content_strs = set()
        content = []
        for mem in relevant_memories:
            if mem.page_content in content_strs:
                continue
            content_strs.add(mem.page_content)
            created_time = mem.metadata["created_at"].strftime("%B %d, %Y, %I:%M %p")
            content.append(f"- {created_time}: {mem.page_content.strip()}")
        return "\n".join([f"{mem}" for mem in content])
    
    def summarize_related_memories(self, observation: str) -> str:
        """Summarize memories that are most relevant to an observation."""
        entity_name = self._get_entity_from_observation(observation)
        entity_action = self._get_entity_action(observation, entity_name)
        q1 = f"What is the relationship between {self.name} and {entity_name}"
        relevant_memories = self.fetch_memories(q1) # Fetch memories related to the agent's relationship with the entity
        q2 = f"{entity_name} is {entity_action}"
        relevant_memories += self.fetch_memories(q2) # Fetch things related to the entity-action pair
        context_str = self._format_memories_to_summarize(relevant_memories)
        prompt = PromptTemplate.from_template(
            "{q1}?\nContext from memory:\n{context_str}\nRelevant context: "
        )
        chain = LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
        return chain.run(q1=q1, context_str=context_str.strip()).strip()

## Functions for interactions

Characters are meant for interactions and actions. Let's define a few functions to facilitate this

In [23]:
def _generate_reaction(
                  agent: GenerativeAgent,
                  observation: str,
                  call_to_action_templ: str,
                  verbose: bool = False
                 ) -> str:
    prompt = PromptTemplate.from_template(
        "{agent_summary_description}"
        +"\nIt is {current_time}."
        +"\n{agent_name}'s status: {agent_status}"
        + "\nObservation: {observation}"
        + "\nSummary of relevant context from {agent_name}'s memory:"
        +"\n{relevant_memories}"
        + "\n" + call_to_action_templ
    )
    agent_summary_description = agent.get_summary()
    relevant_memories_str = agent.summarize_related_memories(observation)
    current_time_str = datetime.now().strftime("%B %d, %Y, %I:%M %p")
    action_prediction_chain = LLMChain(llm=agent.llm, prompt=prompt)
    kwargs = dict(agent_summary_description=agent_summary_description,
                  current_time=current_time_str,
                  relevant_memories=relevant_memories_str,
                  agent_name=agent.name,
                  observation=observation,
                 agent_status=agent.status)
    result = action_prediction_chain.run(**kwargs)
    return result.strip()

def generate_reaction(agent, observation) -> Tuple[bool, str]:
    call_to_action_template = (
        "Should {agent_name} react to the observation, and if so,"
        +" what would be an appropriate reaction? Respond in one line."
        +' If the action is to engage in dialogue, write:\nSAY: "what to say"'
        +"\notherwise, write:\nREACT: {agent_name}'s reaction (if anything).\nEither do nothing, react, or say something but not both.\n\n"
    )
    full_result = _generate_reaction(agent, observation, call_to_action_template)
    result = full_result.strip().split('\n')[0]
    agent.add_memory(f"{agent.name} observed {observation} and reacted by {result}")
    if "REACT:" in result:
        reaction = result.split("REACT:")[-1].strip()
        return False, reaction
    if "SAY:" in result:
        return True, result.split("SAY:")[-1].strip()
    else:
        raise ValueError("Invalid response: ", result)

def generate_dialogue_response(agent, observation: str):
    call_to_action_template = (
        'What would {agent_name} say? Respond in 1 line. To end the conversation, write:\nGOODBYE: "what to say as a farewell"\nOtherwise to continue the conversation, write:\nSAY: "what to say next"\n\n'
    )
    full_result = _generate_reaction(agent, observation, call_to_action_template)
    result = full_result.strip().split('\n')[0]
    if "GOODBYE:" in result:
        farewell = result.split("GOODBYE:")[-1].strip()
        agent.add_memory(f"{agent.name} observed {observation} and said {farewell}")
        return False, farewell
    if "SAY:" in result:
        response_text = result.split("SAY:")[-1].strip()
        agent.add_memory(f"{agent.name} observed {observation} and said {response_text}")
        return True, response_text
    else:
        raise ValueError("Invalid response: ", full_result)
        
def interview_agent(agent, message: str):
    new_message = f"Interviewer says {message}"
    return generate_dialogue_response(agent, new_message)[1]

## Create a Character

For this example, we will create two characters named "Tommie" and "Eve".

In [24]:
import faiss
def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = OpenAIEmbeddings()
    # Initialize the vectorstore as empty
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})
    return TimeWeightedVectorStoreRetriever(vectorstore=vectorstore, other_score_keys=["importance"])    

In [ ]:
tommie = GenerativeAgent(name="Tommie", 
              age=25,
              traits="anxious, likes design", # You can add more persistent traits here 
              status="looking for a job", # When connected to a virtual world, we can have the characters update their status
              memory_retriever=create_new_memory_retriever(),
              llm=LLM,
              daily_summaries = [
                   "Drove across state to move to a new town but doesn't have a job yet."
               ],
               reflection_threshold = 10, # we will give this a relatively low number to show how reflection works
             )

In [26]:
# We can see a current "Summary" of a character based on their own perception of self.
# It isn't very complete right now since the character doesn't have any memories.
print(tommie.get_summary())

Name: Tommie (age: 25)
Innate traits: anxious, likes design
No statements provided.


In [27]:
# We can give the character memories directly
tommie_memories = [
    "Tommie remembers his dog, Bruno, from when he was a kid",
    "Tommie feels tired from driving so far",
    "Tommie sees the new home",
    "The new neighbors have a cat",
    "The road is noisy at night",
    "Tommie is hungry",
    "Tommie tries to get some rest.",
]
for memory in tommie_memories:
    tommie.add_memory(memory)

In [28]:
# Now that Tommie has 'memories', their self-summary is more descriptive
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design
Tommie is observant, has memories of his past, is affected by environmental factors such as noise, and has basic physical needs like rest and food.


## Pre-Interview with Character

Before sending our character on their way, let's ask them a few questions.

In [29]:
interview_agent(tommie, "What do you like to do?")

'"I enjoy design and creating visually appealing content."'

In [30]:
interview_agent(tommie, "What are you looking forward to doing today?")

'"I\'m looking forward to exploring new design opportunities."'

In [31]:
interview_agent(tommie, "What are you most worried about today?")

Character Tommie is reflecting


'"I\'m most worried about finding a job that allows me to use my design skills to their fullest potential."'

## Step through the day's observations.

In [33]:
# Let's have Tommie start going through a day in the life.
observations = [
    "Tommie wakes up to the sound of a noisy construction site outside his window.",
    "Tommie gets out of bed and heads to the kitchen to make himself some coffee.",
    "Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some.",
    "Tommie finally finds the filters and makes himself a cup of coffee.",
    "The coffee tastes bitter, and Tommie regrets not buying a better brand.",
    "Tommie checks his email and sees that he has no job offers yet.",
    "Tommie spends some time updating his resume and cover letter.",
    "Tommie heads out to explore the city and look for job openings.",
    "Tommie sees a sign for a job fair and decides to attend.",
    "The line to get in is long, and Tommie has to wait for an hour.",
    "Tommie meets several potential employers at the job fair but doesn't receive any offers.",
    "Tommie leaves the job fair feeling disappointed.",
    "Tommie stops by a local diner to grab some lunch.",
    "The service is slow, and Tommie has to wait for 30 minutes to get his food.",
    "Tommie overhears a conversation at the next table about a job opening.",
    "Tommie asks the diners about the job opening and gets some information about the company.",
    "Tommie decides to apply for the job and sends his resume and cover letter.",
    "Tommie continues his search for job openings and drops off his resume at several local businesses.",
    "Tommie takes a break from his job search to go for a walk in a nearby park.",
    "A dog approaches and licks Tommie's feet, and he pets it for a few minutes.",
    "Tommie sees a group of people playing frisbee and decides to join in.",
    "Tommie has fun playing frisbee but gets hit in the face with the frisbee and hurts his nose.",
    "Tommie goes back to his apartment to rest for a bit.",
    "A raccoon tore open the trash bag outside his apartment, and the garbage is all over the floor.",
    "Tommie takes a nap but has trouble sleeping because of the noisy construction site outside his window.",
    "Tommie wakes up feeling even more tired than before.",
    "Tommie decides to watch some TV to relax.",
    "The show he's watching is a repeat, and he gets bored after a few minutes.",
    "Tommie sees a commercial for a new energy drink and decides to try it.",
    "The energy drink tastes terrible, and Tommie regrets buying it.",
    "Tommie starts to feel frustrated with his job search.",
    "Tommie calls his best friend to vent about his struggles.",
    "Tommie's friend offers some words of encouragement and tells him to keep trying.",
    "Tommie feels slightly better after talking to his friend.",
    "Tommie decides to go for a jog to clear his mind.",
    "Tommie jogs through the city and sees some interesting sights.",
    "Tommie stops to take a picture of a street mural.",
    "Tommie runs into an old friend from college who now lives in the city.",
    "They catch up for a few minutes, but Tommie's friend has to leave to attend a meeting.",
    "Tommie thanks his friend and feels hopeful again.",
    "Tommie heads back to his apartment to rest and prepare for his upcoming interviews.",
    "Tommie spends the evening rehearsing his interview pitch."
]


In [35]:
# Let's send Tommie on their way. We'll check in on their summary every few observations to watch it evolve
for i, observation in enumerate(observations):
    _, reaction = generate_reaction(tommie, observation)
    print(f"Tommie observed {observation} and reacted with: {reaction}")
    if ((i+1) % 10) == 0:
        print(f"After {i+1} observations, Tommie's summary is:\n{tommie.get_summary()}")

NameError: name 'reaction' is not defined

In [ ]:
eve = GenerativeAgent(name="Eve", 
              age=34, 
              traits="N/A", # You can add more persistent traits here 
              status="N/A", # When connected to a virtual world, we can have the characters update their status
              memory_retriever=create_new_memory_retriever(),
              llm=LLM,
              daily_summaries = [
                  ("Eve started her new job as a career counselor last week and received her first assignment, a client named Tommie.")
              ],
             )

In [ ]:
yesterday = (datetime.now() - timedelta(days=1)).strftime("%A %B %d")
eve_memories = [
    "Eve overhears her colleague say something about a new client being hard to work with",
    "Eve wakes up and hear's the alarm",
    "Eve eats a boal of porridge",
    "Eve helps a coworker on a task",
    "Eve plays tennis with her friend Xu before going to work",
    "Eve overhears her colleague say something about Tommie being hard to work with",
    
]
for memory in eve_memories:
    eve.add_memory(memory)

In [ ]:
print(eve.get_summary())

## Planning

The authors of ...LINK HERE... recognize that simply prompting with current time,
agent name/traits, and "what would you do now" would result in temporal
redundancies/inconsistencies. Much better to develop a full plan of the day. 

We haven't run ablations on how helpful the recursive plan refinement is, but this is meant to be similar to the process described in the paper:
1. Generate a coarse plan of the day based on the agent's "qualities", yesterday's activities, and the current day.
2. Refine each step to an hour-by-hour plan.
3. Refine each hourly step to 5-15 minute intervals.


In [ ]:
def _create_coarse_plan(agent: GenerativeAgent, verbose: bool = False) -> List[str]:
    previous_day_summary = agent.daily_summaries[-1]
    current_date = datetime.now()
    prompt = PromptTemplate.from_template(
        "Name: {name} (age: {age})"
        +"\nInnate traits: {innate_traits}"
        + '\nOn {previous_day}, {name}'
        + " {previous_day_summary}"
        + '\nToday is {current_day}.'
        + " Here is {name}'s plan today"
        + " in broad strokes:"
        + "\n1. "
    )
    previous_day = (current_date - timedelta(days=1)).strftime("%A %B %d")
    current_day = current_date.strftime("%A %B %d")
    coarse_planner = LLMChain(llm=agent.llm, prompt=prompt, verbose=verbose)
    result = coarse_planner.run(name=agent.name,
                                current_day=current_day,
                                age=agent.age,
                                innate_traits=agent.traits,
                                previous_day=previous_day,
                                previous_day_summary=previous_day_summary
                               )
    return self._parse_list(result)

def _decompose_step(agent: GenerativeAgent, high_level_plan: List[str], target_granularity: str, verbose: bool = False) -> List[str]:
    prompt = PromptTemplate.from_template(
        "Today's High Level Plan: {high_level_plan}"
        + '\nBelow is a detailed plan for the whole day broken into {target_granularity}'
        + "-long chunks:"
        + "\n1. "
    )
    decomposing_planner = LLMChain(llm=agent.llm, prompt=prompt, verbose=verbose)
    high_level_plan_str = " ".join(high_level_plan)
    result = decomposing_planner.run(
        high_level_plan=high_level_plan_str,
        target_granularity=target_granularity,
    )
    return self._parse_list(result)

def _decompose_plan(
        agent: GenerativeAgent,
        high_level_plan: List[str],
        granularities: List[str], # hour, 5-15 minute
        verbose: bool = False
    ):
    result = high_level_plan
    for granularity in granularities:
        # TODO: We could recurse on each step but then there may end up being
        # a need to rectify between windows
        result = _decompose_step(agent, result, granularity, verbose=verbose)
    return result

def generate_full_plan(agent: GenerativeAgent, verbose: bool = False) -> List[str]:
    coarse_plan = _create_coarse_plan(agent)
    today = datetime.now()
    # Save to memory stream
    agent.add_memory(f"{agent.name}'s initial plan for {today.strftime('%A %B %d')}: {' '.join(coarse_plan)}")
    full_plan = _decompose_plan(
        agent,
        coarse_plan, 
        ["hour", "5-15 minute"],
        verbose=verbose,
    )
    # Save to memory stream
    # Unsure if this is done
    agent.add_memory(f"{agent.name}'s full plan for {today.strftime('%A %B %d')}: {' '.join(full_plan)}")
    return full_plan

def update_todays_plan(agent: GenerativeAgent, full_plan: List[str], observation: str, reaction: str, verbose: bool = False) -> List[str]:
    """When the agent makes a reaction to an observation, update their daily plan if needed."""
    prompt = PromptTemplate.from_template(
        "{agent_summary_description}"
        +"\nIt is {current_time}."
        +"\n{name} observed {observation} and had the following reaction: {reaction}."
        +"\n{name}'s original plan for the rest of the day:\n"
        +"{original_plan}"
        +"\nIf this reaction influences the plan, respond with the updated plan at a 5-15 minute granularity. If not, return the original plan unchanged.\n"
        +"(Possibly) Updated Plan:\n\n"
        )
    chain = LLMChain(llm=llm, prompt=prompt, verbose=verbose)
    current_time_str = datetime.now().strftime("%B %d, %Y, %I:%M %p")
    already_completed = []
    original_plan = []
    for item in full_plan:
        split_item = item.split(' - ')
        time_format = '%I:%M%p'
        time_obj = datetime.strptime(split_item[0], time_format)
        if time_obj.time() > datetime.now().time():
            original_plan.append(item)
        else:
            already_completed.append(item)
    current_plan_str = "\n".join([f"{plan}" for plan in original_plan])
    result = chain.run(
        agent_summary_description=agent.get_summary(), 
        current_time = current_time_str,
        name=agent.name,
        observation=observation,
        reaction=reaction,
        original_plan=current_plan_str)
    new_plan = self._parse_list(result)
    return already_completed + new_plan
    

In [ ]:
tommie_full_plan = generate_full_plan(tommie)
tommie_full_plan

In [ ]:
eve_full_plan = generate_full_plan(eve)
eve_full_plan

## Acting in the environment


Agents can plan and act based on obserations, plans, and personalities.

## Environment 

## GenerativeAgent Dialogue

## Pre-conversation interviews


Let's "Interview" our characters before their conversation

In [ ]:
interview_agent(tommie, "How are you feeling about today's plan?")

In [ ]:
interview_agent(tommie, "How do you feel about Eve?")

In [ ]:
interview_agent(tommie, "What are looking forward to today?")

**Now let's "interview" Eve**

In [ ]:
interview_agent(eve, "How are you feeling about today's plan?")

In [ ]:
interview_agent(eve, "How do you feel about Tommie?")

In [ ]:
interview_agent(eve, "What are looking forward to today?")

### First dialogue

In [ ]:
def run_conversation(agents, initial_observation: str, max_turns:int = 25, reflection_frequency: Optional[int] = 5):
    _, observation = generate_reaction(agents[1], initial_observation)
    turns = 0
    while True:
        if reflection_frequency is not None:
            if (turns + 1) % reflection_frequency == 0:
                print("*" * 80)
                print(colored("Reflecting", "blue"))
                for agent in agents:
                    reflections = "\n".join([f" - {reflection}" for reflection in agent.pause_to_reflect()])
                    print(colored(f"{agent.name}'s reflections:", "green"))
                    print(colored(f"{reflections}", "blue"))
                print("*" * 80)
        break_dialogue = False
        for agent in agents:
            stay_in_dialogue, reaction = generate_dialogue_response(agent, observation)
            print(agent.name, reaction)
            observation = f"{agent.name} said {reaction}"
            if not stay_in_dialogue:
                break_dialogue = True   
        if break_dialogue:
            break
        turns += 1


In [ ]:
agents = [tommie, eve]
run_conversation(agents, "Tommie sits down in front of Eve's desk.")

## Let's interview our agents after their conversation

In [ ]:
# We can see how the agents have 
print(tommie.get_summary(force_refresh=True))

# We can see a current "Summary" of a character based on their own perception of self
print(eve.get_summary(force_refresh=True))

In [ ]:
generate_dialogue_response(tommie, "How do you feel about Eve?")[1]

In [ ]:
generate_dialogue_response(tommie, "What are looking forward to today?")[1]

In [ ]:
generate_dialogue_response(eve, "How do you feel about Tommie?")[1]

In [ ]:
generate_dialogue_response(eve, "What are looking forward to today?")[1]

### Characters can respond to input